In [35]:
import re

In [36]:
test_string = """Monkey 0:
  Starting items: 79, 98
  Operation: new = old * 19
  Test: divisible by 23
    If true: throw to monkey 2
    If false: throw to monkey 3

Monkey 1:
  Starting items: 54, 65, 75, 74
  Operation: new = old + 6
  Test: divisible by 19
    If true: throw to monkey 2
    If false: throw to monkey 0

Monkey 2:
  Starting items: 79, 60, 97
  Operation: new = old * old
  Test: divisible by 13
    If true: throw to monkey 1
    If false: throw to monkey 3

Monkey 3:
  Starting items: 74
  Operation: new = old + 3
  Test: divisible by 17
    If true: throw to monkey 0
    If false: throw to monkey 1"""

In [37]:

from dataclasses import dataclass


@dataclass
class Monkey:
    id: str = None,
    items: list = None,
    throw_to: dict = None,
    op: str = None,
    operand: str = None
    modulus: int = None
    others: dict = None
    inspect_count: int = 0
    
    def check(self):
        return self.id is not None and \
               self.items is not None and \
               self.throw_to is not None and \
               self.op is not None and \
               self.operand is not None and \
               self.modulus is not None
               
    def inspect(self, item):
        if self.operand == "old":
            if self.op == "+":
                return item*2
            if self.op == "*":
                return item**2
        else:
            if self.op == "+":
                return item+int(self.operand)
            if self.op == "*":
                return item*int(self.operand)
            
    def inspect_all(self):
        self.inspect_count += len(self.items)
        for i in range(len(self.items)):
            # print(f"Monkey {self.id} inspects {self.items[i]}, new worry {self.inspect(self.items[i]) // 3}")
            self.items[i] = self.inspect(self.items[i]) // 3
    
    def throw_item(self, item):
        other = self.others[self.throw_to[item % self.modulus == 0]]
        print(f"Monkey {self.id} throws {item} at Monkey {other.id}")
        other.items.append(item)
        
    def monkey_round(self):
        self.inspect_all()
        self.items.reverse()
        while len(self.items) > 0:
            self.throw_item(self.items.pop())
            
               


In [38]:
def parse(input: str) -> dict[Monkey]:
    monkey_buffer = None
    monkeys = {}
    for line in input.splitlines():
        line = line.strip()
        if len(line) == 0: continue
        if line.startswith("Monkey"):
            if monkey_buffer is not None: 
                if monkey_buffer.check():
                    monkeys[monkey_buffer.id] = monkey_buffer
                else:
                    raise ValueError(f"Malformed Monkey {monkey_buffer.id}")
            monkey_buffer = Monkey()
            monkey_buffer.id = int(re.search("Monkey ([\d]+):", line).groups()[0])
            monkey_buffer.items = []
            monkey_buffer.throw_to = {}
            monkey_buffer.others = monkeys
            continue
        if line.startswith("Starting items"):
            monkey_buffer.items = [int(x) for x in (re.search("Starting items: (.+)", line).groups()[0].split(","))]
            continue
        if line.startswith("Operation:"):
            op, operand = re.search("Operation: new = old (.) (.+)", line).groups()
            monkey_buffer.op = op
            monkey_buffer.operand = operand
            continue
        if line.startswith("Test"):
            monkey_buffer.modulus = int(re.search("Test: divisible by (\d+)", line).groups()[0])
            continue
        if line.startswith("If true"):
            monkey_buffer.throw_to[True] = int(re.search("If true: throw to monkey (\d+)", line).groups()[0])
            continue
        if line.startswith("If false"):
            monkey_buffer.throw_to[False] = int(re.search("If false: throw to monkey (\d+)", line).groups()[0])
            continue
        raise ValueError(f"Bad input: {line}")
    if monkey_buffer is not None: 
        if monkey_buffer.check():
            monkeys[monkey_buffer.id] = monkey_buffer
        else:
            raise ValueError(f"Malformed Monkey {monkey_buffer.id}")
    return monkeys
        

        

In [39]:
# monkeys = parse(open("../inputs/11.txt").read())
monkeys = parse(test_string)
for i in range(20):
    print(f"round {i}")
    for monkey in monkeys.values():
        monkey.monkey_round()

activity = sorted([monkey.inspect_count for monkey in monkeys.values()], reverse=True)
activity[0] * activity[1]

round 0
Monkey 0 throws 500 at Monkey 3
Monkey 0 throws 620 at Monkey 3
Monkey 1 throws 20 at Monkey 0
Monkey 1 throws 23 at Monkey 0
Monkey 1 throws 27 at Monkey 0
Monkey 1 throws 26 at Monkey 0
Monkey 2 throws 2080 at Monkey 1
Monkey 2 throws 1200 at Monkey 3
Monkey 2 throws 3136 at Monkey 3
Monkey 3 throws 25 at Monkey 1
Monkey 3 throws 167 at Monkey 1
Monkey 3 throws 207 at Monkey 1
Monkey 3 throws 401 at Monkey 1
Monkey 3 throws 1046 at Monkey 1
round 1
Monkey 0 throws 126 at Monkey 3
Monkey 0 throws 145 at Monkey 3
Monkey 0 throws 171 at Monkey 3
Monkey 0 throws 164 at Monkey 3
Monkey 1 throws 695 at Monkey 0
Monkey 1 throws 10 at Monkey 0
Monkey 1 throws 57 at Monkey 2
Monkey 1 throws 71 at Monkey 0
Monkey 1 throws 135 at Monkey 0
Monkey 1 throws 350 at Monkey 0
Monkey 2 throws 1083 at Monkey 3
Monkey 3 throws 43 at Monkey 1
Monkey 3 throws 49 at Monkey 1
Monkey 3 throws 58 at Monkey 1
Monkey 3 throws 55 at Monkey 1
Monkey 3 throws 362 at Monkey 1
round 2
Monkey 0 throws 4401 at

10605